# Ipynb to 

create unique (X,Y) set to train our classifiers and save it to disk using the package `Graphlab` and `SFrames`.

A partir de las 3 tablas ya procesdas : 
* homeantennas
* sumlinks
* groundtruth

armames un datset para explorar conceptos de ML

In [9]:
## seteamos los parametros del notebook
%autosave 180
import pandas as pd; 
import os;
import random;
import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np; import os;import random;

from IPython.display import display # para pretty-print con estetica ipython si es que estamos dentro de  un loop, if, etc

import graphlab as gl
#esto es para dibujar directo a la notebook
# gl.canvas.set_target('ipynb')

# for nice graphics and plots
import seaborn as sns
palette = sns.color_palette()

# for nice long graphic titles
from textwrap import wrap

#seteamos el lugar de trabajo
HOMEDIR=os.path.expanduser('~')

PROJECTDIR = os.getcwd().split(os.sep)
PROJECTDIR =  os.sep.join(PROJECTDIR[:PROJECTDIR.index('mexico-scripts-ver2') + 1])

DATADIR = os.path.join(PROJECTDIR,'datasets')
DATADIR2 = os.path.join(PROJECTDIR,'data')
# os.chdir(DATADIR)


Autosaving every 180 seconds


In [2]:
## set our plotting style
plt.style.use('seaborn-paper')

In [3]:
!ls $DATADIR

data_balanced_sample.csv  gtruth_0215_0715  sl	       tree.dot
gtruth_0114_0715	  homeant	    sl.csv.gz


In [4]:
graphlab_frame = True
# graphlab_frame = 'juan' in HOMEDIR

def get_input_sframe(ds,graphlab_frame=graphlab_frame):
    if ds == 'home_antenna': url = DATADIR +"/homeant"
    elif ds == 'sum_links': url  = DATADIR +"/sl"
    elif ds == 'gtruth_02': url  = DATADIR +"/gtruth_0215_0715" # can be near ground truth (previous to july 2015)
    elif ds == 'gtruth_01': url  =  DATADIR +"/gtruth_0114_0715" # or can be old GT
    else: print('type chosen is %s, type should be home_antenna, sum_links, gtruth_02 or gtruth_01' % ds)
    return url 

In [15]:
DATADIR

'/home/juan/mobility-study/mexico-scripts-ver2/datasets'

In [14]:
!ls $DATADIR

data_balanced_sample.csv  gtruth_0215_0715  sl	       tree.dot
gtruth_0114_0715	  homeant	    sl.csv.gz


In [17]:
%%time
sf_table = gl.load_sframe(get_input_sframe('home_antenna',graphlab_frame))

sl_table = gl.load_sframe(get_input_sframe('sum_links',graphlab_frame))

# no need to sample the gt table since it is very small in memory
gt_table = gl.load_sframe(get_input_sframe('gtruth_01',graphlab_frame))

CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 92.9 ms


### rename ground truth columns

In [18]:
rename_gt = (dict([(col,col+"_gt") for col in gt_table.column_names() if col != 'USER']))

#agrego la etiqueta "_gt" a las columnas del ground_truth
gt_table.rename(rename_gt) 

USER,ANTENNA_ID_0_gt,ANTENNA_ID_WEEKNIGHT_0_gt
41D91B8CB8747902A80D6108D548A8AF ...,2898,2898
24A624FC979DF41830A31C7CED65E2D2 ...,4285,4285
C3FF4C9281746CAC6C09802C254BDE71 ...,4336,4336
212D3A9745F1347063267844311B31FF ...,1200,884
544695294E816F26691C4D01516ABF9B ...,195,195
FEC4BD059A42FB5283E07539C1FD2CB1 ...,262,262
BDEF57DB3EF1035FF3F210B47FC2A5B0 ...,1557,4021
3AE1DCA333E55CA87E4F5331035DB600 ...,608,608
2A72E07E48667A6289D93874A623C152 ...,2302,2302
C5287E4004BE8F3A12AD489FE8204056 ...,2644,828


In [19]:
print(sf_table.shape, sl_table.shape, gt_table.shape)


((2035391, 43), (1787143, 131), (2341136, 3))


## simple format description

Aca miramos las top 10 antenas que utilizo un usuario, ordeandas de 0 a 10 segun el uso, en numero de llamados, que el user le daba a c. antenna. _0_ Seria la antenna mas utilizada y _10_ la menos utilizada. El count es la cantidad de veces que utilizo esa antenna. 

Cuando aparece _WEEKNIGHT_ en el nombre de la columna, es porque cumple la condicion de que los llamados fueron hechos de noche fuera del horario [8,20] y dentro de la semana laboral.

Siguiendo las definiciones del trabajo de Caro, un user es _EPIDEMIC_ siii su ANTENNA_WEEKNIGHT_0 (esta es la home_antenna) pertence a la zona epidemica.

El mobility_diameter es el radio de las antennas (0 si uso una sola, etc.) utilizadas por este user. Nuevamente el modificador _WEEKNIGHT_ solo aplica para antennas utilizadas en esos horarios.

In [20]:
# column names
print(sf_table.column_names())

['USER', 'COUNT_0', 'COUNT_1', 'COUNT_2', 'COUNT_3', 'COUNT_4', 'COUNT_5', 'COUNT_6', 'COUNT_7', 'COUNT_8', 'COUNT_9', 'ANTENNA_ID_0', 'ANTENNA_ID_1', 'ANTENNA_ID_2', 'ANTENNA_ID_3', 'ANTENNA_ID_4', 'ANTENNA_ID_5', 'ANTENNA_ID_6', 'ANTENNA_ID_7', 'ANTENNA_ID_8', 'ANTENNA_ID_9', 'COUNT_WEEKNIGHT_0', 'COUNT_WEEKNIGHT_1', 'COUNT_WEEKNIGHT_2', 'COUNT_WEEKNIGHT_3', 'COUNT_WEEKNIGHT_4', 'COUNT_WEEKNIGHT_5', 'COUNT_WEEKNIGHT_6', 'COUNT_WEEKNIGHT_7', 'COUNT_WEEKNIGHT_8', 'COUNT_WEEKNIGHT_9', 'ANTENNA_ID_WEEKNIGHT_0', 'ANTENNA_ID_WEEKNIGHT_1', 'ANTENNA_ID_WEEKNIGHT_2', 'ANTENNA_ID_WEEKNIGHT_3', 'ANTENNA_ID_WEEKNIGHT_4', 'ANTENNA_ID_WEEKNIGHT_5', 'ANTENNA_ID_WEEKNIGHT_6', 'ANTENNA_ID_WEEKNIGHT_7', 'ANTENNA_ID_WEEKNIGHT_8', 'ANTENNA_ID_WEEKNIGHT_9', 'MOBILITY_DIAMETER', 'MOBILITY_DIAMETER_WEEKNIGHT']


## sum links description

La tabla sum_links tiene mas atributos y con data mas rica en informacion:
    
Esencialmente para c/ user miramos la cantidad de llamados y el tiempo que duraron esos llamados pero segmentando con distintos modificadores. `Time` representa la duracion del llamado y Calls el conteo de llamados.

Los modficadores/ segmentaciones son:

* mes en el cual estamos parados (12 == diciembre, ..., 08 == agosto)
* OUT/IN, separa por la direccion de los llamados si salientes u entrantes.
* _VULN_ : separa los llamados que fueron realizados hacia/desde un target_user (en una llamada hay 2 usuarios, el origin o el target) viviendo en una zona epidemica. Donde la home antena de un target_user determina su vulnerabilidad segun si es zona epidemica o no.
* Weekend, WeekDay y WeekNight son lo que suenan. Weekend el finde, Weeknight la semana pero fuera de horario laboral y Weekday en horario laboral y de lunes a viernes.

Hay solo una columna que no entra enteramente en este esquema que es VULNERABLE. Esta columna hace un conteo p/c/ usuario d cuantos target_users viven en una zona epidemica. Tambien se segmenta esta columna con los modficiadores anteriores (el mes y el out/in).

In [21]:
# column names
print(sl_table.column_names())

['USER', 'TimeWeekEnd_OUT_12', 'CallsWeekDay_OUT_12', 'CallsWeekNight_OUT_12', 'TimeWeekDay_OUT_12', 'CallsWeekEnd_OUT_12', 'TimeWeekNight_OUT_12', 'TimeWeekEnd_OUT_VUL_12', 'CallsWeekDay_OUT_VUL_12', 'CallsWeekNight_OUT_VUL_12', 'TimeWeekDay_OUT_VUL_12', 'CallsWeekEnd_OUT_VUL_12', 'TimeWeekNight_OUT_VUL_12', 'VULNERABLE_OUT_12', 'TimeWeekEnd_IN_12', 'CallsWeekDay_IN_12', 'CallsWeekNight_IN_12', 'TimeWeekDay_IN_12', 'CallsWeekEnd_IN_12', 'TimeWeekNight_IN_12', 'TimeWeekEnd_IN_VUL_12', 'CallsWeekDay_IN_VUL_12', 'CallsWeekNight_IN_VUL_12', 'TimeWeekDay_IN_VUL_12', 'CallsWeekEnd_IN_VUL_12', 'TimeWeekNight_IN_VUL_12', 'VULNERABLE_IN_12', 'TimeWeekEnd_OUT_08', 'CallsWeekDay_OUT_08', 'CallsWeekNight_OUT_08', 'TimeWeekDay_OUT_08', 'CallsWeekEnd_OUT_08', 'TimeWeekNight_OUT_08', 'TimeWeekEnd_OUT_VUL_08', 'CallsWeekDay_OUT_VUL_08', 'CallsWeekNight_OUT_VUL_08', 'TimeWeekDay_OUT_VUL_08', 'CallsWeekEnd_OUT_VUL_08', 'TimeWeekNight_OUT_VUL_08', 'VULNERABLE_OUT_08', 'TimeWeekEnd_IN_08', 'CallsWeekDay_

# Para la
tabla gt (ground_truth) es mas simple la explicacion. Solo se busco el antenna_ID_0 (nuevamente la antenna mas utilizada) por un user

In [22]:
print(gt_table.head())

+-------------------------------+-----------------+---------------------------+
|              USER             | ANTENNA_ID_0_gt | ANTENNA_ID_WEEKNIGHT_0_gt |
+-------------------------------+-----------------+---------------------------+
| 41D91B8CB8747902A80D6108D5... |       2898      |            2898           |
| 24A624FC979DF41830A31C7CED... |       4285      |            4285           |
| C3FF4C9281746CAC6C09802C25... |       4336      |            4336           |
| 212D3A9745F134706326784431... |       1200      |            884            |
| 544695294E816F26691C4D0151... |       195       |            195            |
| FEC4BD059A42FB5283E07539C1... |       262       |            262            |
| BDEF57DB3EF1035FF3F210B47F... |       1557      |            4021           |
| 3AE1DCA333E55CA87E4F533103... |       608       |            608            |
| 2A72E07E48667A6289D93874A6... |       2302      |            2302           |
| C5287E4004BE8F3A12AD489FE8... |       

# We need antennas metadata
to get the epidemicity of each antenna and add that info to the dataset

In [24]:
DATADIR2,DATADIR

('/home/juan/mobility-study/mexico-scripts-ver2/data',
 '/home/juan/mobility-study/mexico-scripts-ver2/datasets')

In [25]:
#agregamos al gt su info de epidemicidad
antennas_file = DATADIR2+'/celdas_limpio.csv'

antennas = gl.SFrame.read_csv(antennas_file, 
                              delimiter= "|", 
                usecols=['LATITUDE','LONGITUDE','CEL_ID','STATE','EPIDEMIC'],
                column_type_hints=[float, float, str,str, bool]
                            )

Finished parsing file /home/juan/mobility-study/mexico-scripts-ver2/data/celdas_limpio.csv

Parsing completed. Parsed 4415 lines in 0.028114 secs.

In [26]:
%%time
# enriquecemos la data de gt con info de epidemicidad

# agrego tambien data de epidemicidad al simpleformat table
sf_table = sf_table.join(antennas['CEL_ID','EPIDEMIC','STATE'], 
                         on = {'ANTENNA_ID_WEEKNIGHT_0':'CEL_ID'},
                         how = 'left')
gt_table = gt_table.join(antennas['CEL_ID','EPIDEMIC','STATE'], 
                         on = {'ANTENNA_ID_WEEKNIGHT_0_gt':'CEL_ID'},
                         how = 'left')

gt_table.rename({'EPIDEMIC':'EPIDEMIC_gt'});

CPU times: user 1min 40s, sys: 3.3 s, total: 1min 43s
Wall time: 22.4 s


# create data matrix

In [27]:
%%time

data = sl_table.join(gt_table['USER','EPIDEMIC_gt'], on = 'USER', how = 'inner')
data = data.join(sf_table, on = 'USER', how = 'inner')
#no podemos tener nulls en el target asi que dropeamos
data = data.dropna(columns = ['EPIDEMIC_gt'], how='any')
del sl_table, sf_table, gt_table

print(data.shape)

(1483010, 176)
CPU times: user 6min 8s, sys: 9.7 s, total: 6min 18s
Wall time: 1min 57s


In [28]:
data['EPIDEMIC_gt'].sum(),data['EPIDEMIC'].sum()

(405473, 401108)

## save output training set

In [34]:
out_path = DATADIR +"/train_set"
data.save(out_path)

In [35]:
data.head()

USER,TimeWeekEnd_OUT_12,CallsWeekDay_OUT_12,CallsWeekNight_OUT_12,TimeWeekDay_OUT_12,CallsWeekEnd_OUT_12
23D263FD33D5B9CEE090AD9587BD5E45 ...,818,11,1,1450,5
6E9B40DE6F44190A2B2D9D83D9932078 ...,1810,32,12,4216,12
BB8CAD1367F30EE7369B60BA756A0EE2 ...,211,20,3,1414,6
9CC0000F7AFDE27A6C685D48C236B98E ...,0,4,0,118,0
E8D6E471AD8705240E223E8F3483486D ...,0,6,0,2083,0
D81B416174A5A9F48FD53D1A713C9CC4 ...,360,24,2,2578,2
A5713009D1BA660A04234A098D81E301 ...,796,14,4,1202,11
B60A95F2D591616A0D41D5232210D06E ...,72,6,1,613,2
035CDD42BCA0FCD8FE7350739FB60635 ...,467,16,18,985,10
AF8129C4F54A90A1308400F6B3610CA1 ...,219,7,2,348,5


In [36]:
data.column_names()

['USER',
 'TimeWeekEnd_OUT_12',
 'CallsWeekDay_OUT_12',
 'CallsWeekNight_OUT_12',
 'TimeWeekDay_OUT_12',
 'CallsWeekEnd_OUT_12',
 'TimeWeekNight_OUT_12',
 'TimeWeekEnd_OUT_VUL_12',
 'CallsWeekDay_OUT_VUL_12',
 'CallsWeekNight_OUT_VUL_12',
 'TimeWeekDay_OUT_VUL_12',
 'CallsWeekEnd_OUT_VUL_12',
 'TimeWeekNight_OUT_VUL_12',
 'VULNERABLE_OUT_12',
 'TimeWeekEnd_IN_12',
 'CallsWeekDay_IN_12',
 'CallsWeekNight_IN_12',
 'TimeWeekDay_IN_12',
 'CallsWeekEnd_IN_12',
 'TimeWeekNight_IN_12',
 'TimeWeekEnd_IN_VUL_12',
 'CallsWeekDay_IN_VUL_12',
 'CallsWeekNight_IN_VUL_12',
 'TimeWeekDay_IN_VUL_12',
 'CallsWeekEnd_IN_VUL_12',
 'TimeWeekNight_IN_VUL_12',
 'VULNERABLE_IN_12',
 'TimeWeekEnd_OUT_08',
 'CallsWeekDay_OUT_08',
 'CallsWeekNight_OUT_08',
 'TimeWeekDay_OUT_08',
 'CallsWeekEnd_OUT_08',
 'TimeWeekNight_OUT_08',
 'TimeWeekEnd_OUT_VUL_08',
 'CallsWeekDay_OUT_VUL_08',
 'CallsWeekNight_OUT_VUL_08',
 'TimeWeekDay_OUT_VUL_08',
 'CallsWeekEnd_OUT_VUL_08',
 'TimeWeekNight_OUT_VUL_08',
 'VULNERABLE_OUT_0